In [45]:
!pip install watson-machine-learning-client-V4

In [51]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [145]:
# Instantiate a client using credentials
wml_credentials = {
      "apikey": "U6Tdly5-huIp8iMjOycw6_Vu9RZcBoOGcJ5PUiWZivPq",
      "instance_id": "27ba5bdb-6d00-41d3-88a8-7dc39ae5945e",
      "url": "https://us-south.ml.cloud.ibm.com",
}

client = WatsonMachineLearningAPIClient(wml_credentials)
client.version

'1.0.55'

In [53]:
!pip install ibm-db

<bound method Connection.close of <ibm_db_dbi.Connection object at 0x7f241d1cc748>>

In [100]:
#Setup Connection to Db2 Instance
import ibm_db
import pandas as pd
import ibm_db_dbi
import numpy


cnn = ibm_db.connect("DATABASE=BLUDB;HOSTNAME=worker2.clustera;PORT=31173;PROTOCOL=TCPIP;UID=user1042;PWD=oJ0T7#_U1-%mb1Wz;", "", "")
hdbi = ibm_db_dbi.Connection(cnn)


In [146]:
#Read the table 
sql_read = 'SELECT * FROM PCSB.RTL_SOUP_LEDGER_TO_WORK'

markets = pd.read_sql(sql_read, hdbi)
markets

,CITY,FCST_DEMAND_1_WK,CURR_INV
0,CITY_ATLANTA,500,200
1,CITY_BOSTON,700,50
2,CITY_CHARLOTTE,100,45
3,CITY_CHICAGO,290,90
4,CITY_COLUMBUS,455,250
5,CITY_DALLAS,1000,100
6,CITY_DENVER,400,150
7,CITY_DETROIT,350,0
8,CITY_HOUSTON,375,25
9,CITY_LAS_VEGAS,675,350


In [147]:
# We don't want the markets to have more material than their forecasted demand.  So capping the max ship
markets['MAX_SHIP'] = markets['FCST_DEMAND_1_WK'] - markets['CURR_INV']
markets.loc[markets['MAX_SHIP'] < 0, 'MAX_SHIP'] = 0
# Total cases avaivlale for shipment is hardcoded for now.  Should really come from an external source


In [148]:
total_shipments = 4320


In [149]:
# Total current inventory, total demand and an overall optimum allocation ratio across all markets
def opt_allow_ratio(tot_shipments): 
    total_curr_inv = markets.CURR_INV.sum()
    total_demand = markets.FCST_DEMAND_1_WK.sum()
    opt_allow_ratio = (tot_shipments + total_curr_inv)/total_demand
    return opt_allow_ratio

In [150]:
opt_allocation_ratio = opt_allow_ratio(total_shipments)
print(opt_allocation_ratio)

0.6482082233119577


In [151]:
from docplex.mp.model import Model

# Model
mdl = Model(name='prs4b')

# Create integer decision variables for shipments.  Lower bound is 0.  Upper bound is forecast - curr inv
qty = markets.copy()
qty['VAR'] = qty.apply(lambda x: mdl.integer_var(name=x['CITY'], ub = x['MAX_SHIP'], lb = 0), axis=1)
# make the name the index
qty.set_index('CITY', inplace=True)

In [152]:
# Create a dummy decision variable representing the diff in allocation ratio.
# Need this dummy dvs to account for the fact that we need to minimize the sum of the absolute
# differences between the allocation ratio in individual markets.  The presence of the 
# absolute number in the objective will make it non-linear. 
# If we have 25 markets, we will have 24 dvs.  DV for market 1 -> 2, DV for market 1 -> 3, 
# market 1 -> 3, etc to 1 - > 25
dummy_dvs = pd.DataFrame({'CITY_A': [qty.index.values[0]]*(qty.shape[0])})
dummy_dvs['CITY_B'] = qty.index.values
dummy_dvs = dummy_dvs.loc[dummy_dvs['CITY_A'] != dummy_dvs['CITY_B']]
dummy_dvs['IDX'] = 'DV_' + dummy_dvs['CITY_A'] + '_' + dummy_dvs['CITY_B']
dummy_dvs['VAR'] = dummy_dvs.apply(lambda x: mdl.continuous_var(name = 'DV_' + x['CITY_A'] + '_' + x['CITY_B']), axis=1)
dummy_dvs.set_index('IDX', inplace = True)

In [153]:
dummy_dvs.head()

,CITY_A,CITY_B,VAR
IDX,,,
DV_CITY_ATLANTA_CITY_BOSTON,CITY_ATLANTA,CITY_BOSTON,DV_CITY_ATLANTA_CITY_BOSTON
DV_CITY_ATLANTA_CITY_CHARLOTTE,CITY_ATLANTA,CITY_CHARLOTTE,DV_CITY_ATLANTA_CITY_CHARLOTTE
DV_CITY_ATLANTA_CITY_CHICAGO,CITY_ATLANTA,CITY_CHICAGO,DV_CITY_ATLANTA_CITY_CHICAGO
DV_CITY_ATLANTA_CITY_COLUMBUS,CITY_ATLANTA,CITY_COLUMBUS,DV_CITY_ATLANTA_CITY_COLUMBUS
DV_CITY_ATLANTA_CITY_DALLAS,CITY_ATLANTA,CITY_DALLAS,DV_CITY_ATLANTA_CITY_DALLAS


In [154]:
# Add constraints
# Total shipment across all markets cannot exceed total available product from suppier
mdl.add_constraint(mdl.sum(qty.loc[m.CITY]['VAR'] for m in markets.itertuples())<= total_shipments, 'CT_Total_Shipment')
mdl.add_constraints( \
                    (((mdl.sum([qty.loc[d[1]]['VAR'], qty.loc[d[1]]['CURR_INV']]))/qty.loc[d[1]]['FCST_DEMAND_1_WK'] - \
                      (mdl.sum([qty.loc[d[2]]['VAR'], qty.loc[d[2]]['CURR_INV']]))/qty.loc[d[2]]['FCST_DEMAND_1_WK']) \
                     <= d[3]) for d in dummy_dvs.itertuples())
mdl.add_constraints( \
                    (-1*((mdl.sum([qty.loc[d[1]]['VAR'], qty.loc[d[1]]['CURR_INV']]))/qty.loc[d[1]]['FCST_DEMAND_1_WK'] - \
                      (mdl.sum([qty.loc[d[2]]['VAR'], qty.loc[d[2]]['CURR_INV']]))/qty.loc[d[2]]['FCST_DEMAND_1_WK']) \
                     <= d[3]) for d in dummy_dvs.itertuples())

[docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.001CITY_BOSTON-0.329,LE,DV_CITY_ATLANTA_CITY_BOSTON),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.010CITY_CHARLOTTE+0.050,LE,DV_CITY_ATLANTA_CITY_CHARLOTTE),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.003CITY_CHICAGO-0.090,LE,DV_CITY_ATLANTA_CITY_CHICAGO),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.002CITY_COLUMBUS+0.149,LE,DV_CITY_ATLANTA_CITY_COLUMBUS),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.001CITY_DALLAS-0.300,LE,DV_CITY_ATLANTA_CITY_DALLAS),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.003CITY_DENVER-0.025,LE,DV_CITY_ATLANTA_CITY_DENVER),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.003CITY_DETROIT-0.400,LE,DV_CITY_ATLANTA_CITY_DETROIT),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.003CITY_HOUSTON-0.333,LE,DV_CITY_ATLANTA_CITY_HOUSTON),
 docplex.mp.LinearConstraint[](-0.002CITY_ATLANTA+0.001CITY_LAS_VEGAS+0.119,LE,DV_CITY_ATLANTA_CITY_LAS_VEGAS),
 docplex.mp.LinearConstr

In [155]:
# Define the objective function 
total_ratio = mdl.sum((-1*(qty.loc[m.CITY]['VAR'] + m.CURR_INV)/m.FCST_DEMAND_1_WK) for m in markets.itertuples())
total_dummy_vars = mdl.sum((d[3]) for d in dummy_dvs.itertuples())
mdl.add_kpi(-1*(total_ratio/25), publish_name="Total Allocation Ratio")
mdl.add_kpi(total_dummy_vars, publish_name="Total Shortage Ratio")
mdl.minimize(total_ratio + total_dummy_vars)

In [156]:
#Creating KPIS
for n in markets.itertuples():
    shortage_ratio_with_var = ((qty.loc[n.CITY]['FCST_DEMAND_1_WK']-(qty.loc[n.CITY]['CURR_INV']))-qty.loc[n.CITY]['VAR'])/qty.loc[n.CITY]['FCST_DEMAND_1_WK']
    
    mdl.add_kpi(shortage_ratio_with_var, publish_name='Shortage percentage %s' % n.CITY)


In [157]:
mdl.print_information()

Model: prs4b
 - number of variables: 49
   - binary=0, integer=25, continuous=24
 - number of constraints: 49
   - linear=49
 - parameters: defaults
 - problem type is: MILP


In [158]:
# solve
ok = mdl.solve()

mdl.print_solution()


objective: -16.195
  CITY_ATLANTA=125
  CITY_BOSTON=405
  CITY_CHARLOTTE=20
  CITY_CHICAGO=99
  CITY_COLUMBUS=46
  CITY_DALLAS=526
  CITY_DENVER=110
  CITY_DETROIT=228
  CITY_HOUSTON=219
  CITY_LAS_VEGAS=89
  CITY_LOS_ANGELES=105
  CITY_MIAMI=184
  CITY_MINNEAPOLIS=220
  CITY_NASHVILLE=197
  CITY_NEW_ORLEANS=53
  CITY_NEW_YORK=77
  CITY_OKLAHOMA_CITY=120
  CITY_PHOENIX=325
  CITY_SACRAMENTO=140
  CITY_SALT_LAKE_CITY=245
  CITY_SAN_DIEGO=82
  CITY_SAN_FRANCISCO=220
  CITY_SEATTLE=212
  CITY_ST_LOUIS=28
  CITY_WICHITA=245
  DV_CITY_ATLANTA_CITY_CHICAGO=0.002
  DV_CITY_ATLANTA_CITY_COLUMBUS=0.001
  DV_CITY_ATLANTA_CITY_DALLAS=0.024
  DV_CITY_ATLANTA_CITY_DETROIT=0.001
  DV_CITY_ATLANTA_CITY_HOUSTON=0.001
  DV_CITY_ATLANTA_CITY_LAS_VEGAS=0.000
  DV_CITY_ATLANTA_CITY_MIAMI=0.001
  DV_CITY_ATLANTA_CITY_NASHVILLE=0.001
  DV_CITY_ATLANTA_CITY_NEW_ORLEANS=0.000
  DV_CITY_ATLANTA_CITY_NEW_YORK=0.001
  DV_CITY_ATLANTA_CITY_SAN_DIEGO=0.001
  DV_CITY_ATLANTA_CITY_SAN_FRANCISCO=0.001
  DV_CITY_ATLAN

In [159]:
mdl.report()

* model prs4b solved with objective = -16.195
*  KPI: Total Allocation Ratio                  = 0.649
*  KPI: Total Shortage Ratio                    = 0.034
*  KPI: Shortage percentage CITY_ATLANTA        = 0.350
*  KPI: Shortage percentage CITY_BOSTON         = 0.350
*  KPI: Shortage percentage CITY_CHARLOTTE      = 0.350
*  KPI: Shortage percentage CITY_CHICAGO        = 0.348
*  KPI: Shortage percentage CITY_COLUMBUS       = 0.349
*  KPI: Shortage percentage CITY_DALLAS         = 0.374
*  KPI: Shortage percentage CITY_DENVER         = 0.350
*  KPI: Shortage percentage CITY_DETROIT        = 0.349
*  KPI: Shortage percentage CITY_HOUSTON        = 0.349
*  KPI: Shortage percentage CITY_LAS_VEGAS      = 0.350
*  KPI: Shortage percentage CITY_LOS_ANGELES    = 0.350
*  KPI: Shortage percentage CITY_MIAMI          = 0.349
*  KPI: Shortage percentage CITY_MINNEAPOLIS    = 0.350
*  KPI: Shortage percentage CITY_NASHVILLE      = 0.351
*  KPI: Shortage percentage CITY_NEW_ORLEANS    = 0.350
* 

In [161]:
solution_df = pd.DataFrame(columns=['CITY', 'SHIPMENT_QTY'])

for index, dvar in enumerate(mdl.solution.iter_variables()):
    if(not dvar.to_string().startswith('DV_')):
        solution_df.loc[index,'CITY'] = dvar.to_string()
        solution_df.loc[index,'SHIPMENT_QTY'] = dvar.solution_value
        solution_df.loc[index,'SCENARIO'] = 'OPTIMIZED'
        solution_df.loc[index,'PERCENT'] = 0
        solution_df.loc[index,'ACTION'] = ''
        solution_df.loc[index, 'MODIFIED_TIMESTAMP'] = pd.Timestamp.now()

#dd-cell
solution_df


,CITY,SHIPMENT_QTY,SCENARIO,PERCENT,ACTION,MODIFIED_TIMESTAMP
0,CITY_ATLANTA,125.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.527930
1,CITY_BOSTON,405.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.537901
2,CITY_CHARLOTTE,20.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.545235
3,CITY_CHICAGO,99.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.551920
4,CITY_COLUMBUS,46.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.558987
5,CITY_DALLAS,526.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.572924
6,CITY_DENVER,110.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.581795
7,CITY_DETROIT,228.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.588960
8,CITY_HOUSTON,219.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.596794
9,CITY_LAS_VEGAS,89.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.604146


In [162]:
subset = pd.DataFrame({'CITY':markets['CITY'], 'FCST_DEMAND_1_WK':markets['FCST_DEMAND_1_WK'],
              'CURR_INV':markets['CURR_INV']})


solutionDF = subset.merge(solution_df, on='CITY')
solutionDF

,CITY,FCST_DEMAND_1_WK,CURR_INV,SHIPMENT_QTY,SCENARIO,PERCENT,ACTION,MODIFIED_TIMESTAMP
0,CITY_ATLANTA,500,200,125.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.527930
1,CITY_BOSTON,700,50,405.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.537901
2,CITY_CHARLOTTE,100,45,20.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.545235
3,CITY_CHICAGO,290,90,99.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.551920
4,CITY_COLUMBUS,455,250,46.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.558987
5,CITY_DALLAS,1000,100,526.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.572924
6,CITY_DENVER,400,150,110.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.581795
7,CITY_DETROIT,350,0,228.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.588960
8,CITY_HOUSTON,375,25,219.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.596794
9,CITY_LAS_VEGAS,675,350,89.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.604146


In [163]:
import numpy as np

solutionDF.CITY = solutionDF.CITY.astype(str).replace('nan',np.nan)
solutionDF.FCST_DEMAND_1_WK = solutionDF.FCST_DEMAND_1_WK.astype(str).replace('nan',np.nan)
solutionDF.CURR_INV = solutionDF.CURR_INV.astype(str).replace('nan',np.nan)
solutionDF.SHIPMENT_QTY = solutionDF.SHIPMENT_QTY.astype(str).replace('nan',np.nan)
solutionDF.SCENARIO = solutionDF.SCENARIO.astype(str).replace('nan',np.nan)
solutionDF.PERCENT = solutionDF.PERCENT.astype(str).replace('nan',np.nan)
solutionDF.ACTION = solutionDF.ACTION.astype(str).replace('nan',np.nan)


solutionDF = solutionDF.where(pd.notnull(solutionDF), None) 
solutionDF.head()

,CITY,FCST_DEMAND_1_WK,CURR_INV,SHIPMENT_QTY,SCENARIO,PERCENT,ACTION,MODIFIED_TIMESTAMP
0,CITY_ATLANTA,500,200,125.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.527930
1,CITY_BOSTON,700,50,405.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.537901
2,CITY_CHARLOTTE,100,45,20.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.545235
3,CITY_CHICAGO,290,90,99.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.551920
4,CITY_COLUMBUS,455,250,46.0,OPTIMIZED,0.0,,2020-04-09 16:11:51.558987


In [164]:
tuple_of_tuples = tuple([tuple(x) for x in solutionDF.values])

sql = "INSERT INTO PCSB.RTL_OPTIMIZED_SHIPMENT VALUES(?,?,?,?,?,?,?,?)"


stmt = ibm_db.prepare(cnn, sql)

ibm_db.execute_many(stmt, tuple_of_tuples)

25

<h3>Defining the Modeling process as a function<h3>

In [165]:
def modeling_writing_to_db(total_shipments, reduction_percent): 
    
    opt_allocation_ratio = opt_allow_ratio(total_shipments)
    

    # Model
    mdl = Model(name='prs4b')

    # Create integer decision variables for shipments.  Lower bound is 0.  Upper bound is forecast - curr inv
    qty = markets.copy()
    qty['VAR'] = qty.apply(lambda x: mdl.integer_var(name=x['CITY'], ub = x['MAX_SHIP'], lb = 0), axis=1)
    # make the name the index
    qty.set_index('CITY', inplace=True)
    print(qty)
    # Create a dummy decision variable representing the diff in allocation ratio.
    # Need this dummy dvs to account for the fact that we need to minimize the sum of the absolute
    # differences between the allocation ratio in individual markets.  The presence of the 
    # absolute number in the objective will make it non-linear. 
    # If we have 25 markets, we will have 24 dvs.  DV for market 1 -> 2, DV for market 1 -> 3, 
    # market 1 -> 3, etc to 1 - > 25
    dummy_dvs = pd.DataFrame({'CITY_A': [qty.index.values[0]]*(qty.shape[0])})
    dummy_dvs['CITY_B'] = qty.index.values
    dummy_dvs = dummy_dvs.loc[dummy_dvs['CITY_A'] != dummy_dvs['CITY_B']]
    dummy_dvs['IDX'] = 'DV_' + dummy_dvs['CITY_A'] + '_' + dummy_dvs['CITY_B']
    dummy_dvs['VAR'] = dummy_dvs.apply(lambda x: mdl.continuous_var(name = 'DV_' + x['CITY_A'] + '_' + x['CITY_B']), axis=1)
    dummy_dvs.set_index('IDX', inplace = True)

    # Add constraints
    # Total shipment across all markets cannot exceed total available product from suppier
    mdl.add_constraint(mdl.sum(qty.loc[m.CITY]['VAR'] for m in markets.itertuples())<= total_shipments, 'CT_Total_Shipment')
    mdl.add_constraints( \
                        (((mdl.sum([qty.loc[d[1]]['VAR'], qty.loc[d[1]]['CURR_INV']]))/qty.loc[d[1]]['FCST_DEMAND_1_WK'] - \
                          (mdl.sum([qty.loc[d[2]]['VAR'], qty.loc[d[2]]['CURR_INV']]))/qty.loc[d[2]]['FCST_DEMAND_1_WK']) \
                         <= d[3]) for d in dummy_dvs.itertuples())
    mdl.add_constraints( \
                        (-1*((mdl.sum([qty.loc[d[1]]['VAR'], qty.loc[d[1]]['CURR_INV']]))/qty.loc[d[1]]['FCST_DEMAND_1_WK'] - \
                          (mdl.sum([qty.loc[d[2]]['VAR'], qty.loc[d[2]]['CURR_INV']]))/qty.loc[d[2]]['FCST_DEMAND_1_WK']) \
                         <= d[3]) for d in dummy_dvs.itertuples())
    # Define the objective function 
    total_ratio = mdl.sum((-1*(qty.loc[m.CITY]['VAR'] + m.CURR_INV)/m.FCST_DEMAND_1_WK) for m in markets.itertuples())
    total_dummy_vars = mdl.sum((d[3]) for d in dummy_dvs.itertuples())
    mdl.add_kpi(-1*(total_ratio/25), publish_name="Total Allocation Ratio")
    mdl.add_kpi(total_dummy_vars, publish_name="Total Shortage Ratio")
    mdl.minimize(total_ratio + total_dummy_vars)

    #Creating KPIS
    for n in markets.itertuples():
        shortage_ratio_with_var = ((qty.loc[n.CITY]['FCST_DEMAND_1_WK']-(qty.loc[n.CITY]['CURR_INV']))-qty.loc[n.CITY]['VAR'])/qty.loc[n.CITY]['FCST_DEMAND_1_WK']

        mdl.add_kpi(shortage_ratio_with_var, publish_name='Shortage percentage %s' % n.CITY)

    print(mdl.print_information())

    # solve
    ok = mdl.solve()

    print(mdl.print_solution())
    print(mdl.report())

    solution_df = pd.DataFrame(columns=['CITY', 'SHIPMENT_QTY'])

    for index, dvar in enumerate(mdl.solution.iter_variables()):
        if(not dvar.to_string().startswith('DV_')):
            solution_df.loc[index,'CITY'] = dvar.to_string()
            solution_df.loc[index,'SHIPMENT_QTY'] = dvar.solution_value
            solution_df.loc[index,'SCENARIO'] = 'OPTIMIZED'
            solution_df.loc[index,'PERCENT'] = reduction_percent
            solution_df.loc[index,'ACTION'] = 'REDUCTION'
            solution_df.loc[index, 'MODIFIED_TIMESTAMP'] = pd.Timestamp.now()

    #dd-cell
    print(solution_df.head())

    subset = pd.DataFrame({'CITY':markets['CITY'], 'FCST_DEMAND_1_WK':markets['FCST_DEMAND_1_WK'],
                  'CURR_INV':markets['CURR_INV']})


    solutionDF = subset.merge(solution_df, on='CITY')
    print(solutionDF.head())

    solutionDF.CITY = solutionDF.CITY.astype(str).replace('nan',np.nan)
    solutionDF.FCST_DEMAND_1_WK = solutionDF.FCST_DEMAND_1_WK.astype(str).replace('nan',np.nan)
    solutionDF.CURR_INV = solutionDF.CURR_INV.astype(str).replace('nan',np.nan)
    solutionDF.SHIPMENT_QTY = solutionDF.SHIPMENT_QTY.astype(str).replace('nan',np.nan)
    solutionDF.SCENARIO = solutionDF.SCENARIO.astype(str).replace('nan',np.nan)
    solutionDF.PERCENT = solutionDF.PERCENT.astype(str).replace('nan',np.nan)
    solutionDF.ACTION = solutionDF.ACTION.astype(str).replace('nan',np.nan)


    solutionDF = solutionDF.where(pd.notnull(solutionDF), None) 
    print(solutionDF.head())

    tuple_of_tuples = tuple([tuple(x) for x in solutionDF.values])

    sql = "INSERT INTO PCSB.RTL_OPTIMIZED_SHIPMENT VALUES(?,?,?,?,?,?,?,?)"


    stmt = ibm_db.prepare(cnn, sql)

    ibm_db.execute_many(stmt, tuple_of_tuples)


<h4>Scenario #2</h4>

In [166]:
print("Scenario 2 where the shipment has been reduced by 20")
modeling_writing_to_db(3456, 20)



Scenario 2 where the shipment has been reduced by 20
                     FCST_DEMAND_1_WK  CURR_INV  MAX_SHIP                  VAR
CITY                                                                          
CITY_ATLANTA                      500       200       300         CITY_ATLANTA
CITY_BOSTON                       700        50       650          CITY_BOSTON
CITY_CHARLOTTE                    100        45        55       CITY_CHARLOTTE
CITY_CHICAGO                      290        90       200         CITY_CHICAGO
CITY_COLUMBUS                     455       250       205        CITY_COLUMBUS
CITY_DALLAS                      1000       100       900          CITY_DALLAS
CITY_DENVER                       400       150       250          CITY_DENVER
CITY_DETROIT                      350         0       350         CITY_DETROIT
CITY_HOUSTON                      375        25       350         CITY_HOUSTON
CITY_LAS_VEGAS                    675       350       325       CITY_LAS_VEGAS

<h4>Scenario #3</h4>

In [167]:
print("Scenario 3 where the shipment has been reduced by 30")
modeling_writing_to_db(3024, 30)

Scenario 3 where the shipment has been reduced by 30
                     FCST_DEMAND_1_WK  CURR_INV  MAX_SHIP                  VAR
CITY                                                                          
CITY_ATLANTA                      500       200       300         CITY_ATLANTA
CITY_BOSTON                       700        50       650          CITY_BOSTON
CITY_CHARLOTTE                    100        45        55       CITY_CHARLOTTE
CITY_CHICAGO                      290        90       200         CITY_CHICAGO
CITY_COLUMBUS                     455       250       205        CITY_COLUMBUS
CITY_DALLAS                      1000       100       900          CITY_DALLAS
CITY_DENVER                       400       150       250          CITY_DENVER
CITY_DETROIT                      350         0       350         CITY_DETROIT
CITY_HOUSTON                      375        25       350         CITY_HOUSTON
CITY_LAS_VEGAS                    675       350       325       CITY_LAS_VEGAS